In [7]:
import sys
import os
import torch
import numpy as np
import yaml
from tqdm import tqdm
from PIL import Image
import glob

from utils_func import *
from sample_func import *
from colorizer import *

In [9]:
# Coco model
ckpt_file = '/home/huangzhitong/code/unicolor/framework/logs/bert_final/epoch=14-step=142124_new.ckpt'
device = 'cuda:0'
colorizer = Colorizer(ckpt_file, device, [256, 256], load_clip=False, load_warper=False)

Working with z of shape (1, 1024, 16, 16) = 262144 dimensions.


### Colorize COCO val set

In [13]:
org_path = '/home/huangzhitong/code/unicolor/results/test/original'
col_path = '/home/huangzhitong/code/unicolor/results/test/colorized'

os.makedirs(org_path, exist_ok=True)
os.makedirs(col_path, exist_ok=True)

img_paths = glob.glob('/home/huangzhitong/dataset/coco/val2017/*.jpg')

for path in tqdm(img_paths[1011:1012]):
    img = Image.open(path).convert('RGB')
    gray = img.convert('L')
    colorized = colorizer.sample(gray, [], topk=100)
    name = path.split('/')[-1].split('.')[0]
    img.save(os.path.join(org_path, name+'.png'))
    colorized.save(os.path.join(col_path, name+'.png'))


100%|██████████| 1/1 [00:03<00:00,  3.45s/it]


### Update old state keys

In [8]:
ckpt_file = '/home/huangzhitong/code/unicolor/framework/logs/bert_final/epoch=14-step=142124.ckpt'
ckpt = torch.load(ckpt_file)
state = ckpt['state_dict']
for key in list(state.keys()):
    state[key.replace('hybrid_vqgan', 'chroma_vqgan').replace('coltran', 'hybrid_tran')] = state.pop(key)
torch.save(ckpt, ckpt_file.replace('.ckpt', '_new.ckpt'))